# Deel II (BASH scripting)

In dit deel gebeurt alles met bash-internals, behalve de opdracht sort die je uitsluitend gebruikt om te
ordenen. Het gebruikt van externe commando’s en ook andere interpreters zoals awk, sed, python, perl,
... is niet toegelaten. Enkel het externe commando sort mag je gebruiken in één van de onderstaande
opgaven om te ordenen/sorteren.

1. [7.5pt] Gegeven het invoerbestand tokyo_metro.txt.

* a) Schrijf een script 1a.bash dat alle namen van de verschillende metrolijnen naar het scherm schrijft. Één naam van een metrolijn per lijn. De uitvoer hoeft niet geordend te zijn.

In [ ]:
%%bash
#!/bin/bash

eerste=0
while IFS=, read naam rest ; do
    ((eerste==0)) && { eerste=1 ; continue ; }
    echo ${naam}
done < tokyo_metro.txt

* b) Schrijf een script 1b.bash dat alle unieke namen van alle metrostations geordend op naam,
naar het scherm schrijft. Eén stationsnaam per lijn. Opgelet, er zijn enkele stations waar er een spatie in de naam te vinden is, bv. Oshiage < SKYTREE >, Nijubashimae < Marunouchi >, .... Die extra uitleg tussen <...> moet er in de uitvoer bij omdat die gebruikt wordt om het onderscheid te maken tussen een station voor een metrolijn en een gewone spoorlijn die voor een ander station helaas dezelfde naam gebruikt. Gebruik gerust het externe commando sort om te ordenen.

In [ ]:
%%bash
#!/bin/bash

eerste=0
declare -A resultaat
while read line ; do
    ((eerste==0)) && { eerste=1 ; continue ; }
    IFS=, read -a array <<< "$line"
    unset array[0]
    unset array[1]
    # quotes om spaties in de array elementen te behouden
    for i in "${array[@]}" ; do
        IFS='[' read station rest <<< "$i"
        resultaat[$station]=1
    done
done < tokyo_metro.txt

for station in "${!resultaat[@]}" ; do
    echo $station
done | sort

Extra opties voor het examen: enkel de verhoogde stations geven, enkel de stations die de eerste x kilometer van de lijn liggen...

In [ ]:
%%bash
#!/bin/bash

eerste=0
declare -A resultaat
while read line ; do
    ((eerste==0)) && { eerste=1 ; continue ; }
    IFS=, read -a array <<< "$line"
    unset array[0]
    unset array[1]
    # quotes om spaties in de array elementen te behouden
    for i in "${array[@]}" ; do
        IFS='[' read station rest <<< "$i"
        IFS='(' read voor km verhoogd <<< "$rest"
        if [[ ${verhoogd%.*} != "Underground" ]] ; then
            echo $station
        fi  
        resultaat[$station]=1
    done
done < tokyo_metro.txt

echo ""
echo "Stations boven de grond (gesorteerd):"

for station in "${!resultaat[@]}" ; do
    echo $station
done | sort

In [ ]:
%%bash
#!/bin/bash

eerste=0
declare -A resultaat
grens=5
while read line ; do
    ((eerste==0)) && { eerste=1 ; continue ; }
    IFS=, read -a array <<< "$line"
    unset array[0]
    unset array[1]
    # quotes om spaties in de array elementen te behouden
    for i in "${array[@]}" ; do
        IFS='[' read station rest <<< "$i"
        IFS='(' read voor km verhoogd <<< "$rest"
        if (( ${km%.*} < $grens )) ; then
            echo $station
        fi  
        resultaat[$station]=1
    done
done < tokyo_metro.txt

for station in "${!resultaat[@]}" ; do
    echo $station
done | sort

Vanaf je ziet staan "het begint met" of "het eindigt op" moet je denken aan **reguliere expressies**

2. [7.5 pt] Schrijf een bash-script met de naam 2.bash. Dit script accepteert verschillende opties en parameters. Het script moet controleren of de ingevoerde opties en parameters geldig zijn. Als alles correct is, moet het script de namen van de gebruikte opties (zonder het voorafgaande minteken) naar het scherm schrijven. In het geval van ongeldige invoer moet het script een foutboodschap weergeven en stoppen met een specifieke exit-status. **Het gebruik van getopt is niet toegelaten!**

Opties en parameters:
* -f: Deze optie moet worden gevolgd door de naam van een bestand. Het script moet
controleren of het bestand bestaat en leesbaar is.
* -s: Deze optie moet worden gevolgd door een getal dat eindigt op KB, MB, of GB, zonder
spaties tussen het getal en de eenheid. Bijvoorbeeld: 10MB, 500GB, 107KB. Het getal moet
groter zijn dan 5. Bijvoorbeeld: 2GB is niet geldig omdat 2 kleiner is dan 5.
* -n, -r, -t: Deze zijn eenvoudige vlaggen zonder extra parameters. Andere opties dan deze
worden niet herkend door het script en moeten een fout veroorzaken.

Bijkomende specificaties:
* De volgorde van de opties maakt niet uit.
* Het script moet niet gevoelig zijn voor hoofdletters of kleine letters. Bijvoorbeeld, -N en -n
moeten als gelijkwaardig worden beschouwd.
* Opties kunnen worden samengevoegd (bijvoorbeeld, -n -r kan worden geschreven als -nr),
maar de -f en -s opties moeten afzonderlijk blijven vanwege hun parameters.

Voorbeelden van correct gebruik:
* 2.bash -n -S 100KB -tr
* 2.bash -f /etc/passwd -nTr
* 2.bash

Voorbeelden van onjuist gebruik:
* 2.bash -s 2TB -p (2 is kleiner dan 5, TB is geen erkende eenheid, en -p is een onbekende
optie)
* 2.bash -nt -f nietleesbaarbestand (het bestand is niet leesbaar)

Foutboodschap:
Als er een fout optreedt, moet het script de volgende boodschap naar het scherm schrijven:
**Usage: 2.bash [-f bestandsnaam] [-s size][-n][-r][-t]**
Let op: Het gebruik van getopt is niet toegestaan in dit script.

In [ ]:
%%bash
#!/bin/bash

while [[ -n "$1" ]] ; do
    case $1 in
        -f)
            [[ ! -f "$2" || ! -r "$2" ]] && {
                echo "Usage 2.bash ..." >&2
                exit 1
            }
            shift
            ;;
        -s)
            [[ $2 =~ KB$ || $2 =~ GB$ || $2 =~ MB$  ]] && {
                echo "Usage 2.bash ..." >&2
                exit 1
            }
            getal = ${2%%??}
            [[ ! $getal =~ ^[1-9][0-9]+$ ]] && ! $getal =~ ^[0-9]$ ]] && {
                echo "Usage 2.bash ..." >&2
                exit 1
            }
            ((getal < 5)) && {
                echo "Usage 2.bash ..." >&2
                exit 1
            }
            shift
            ;;
        -*)
            temp=${1#?}
            while read -N1 kar ; do
                [[ $kar != $'\n' ]] && {
                    [[ $kar != "n" && $kar != "r" && $kar != "t" ]] && {
                        echo "Usage 2.bash ..." >&2
                        exit 1
                    }
                }
            done <<< ${temp,,}
            ;;
    esac
    shift
done 

# Deel III (systeemaanroepen vanuit de programmeertaal C)

In dit deel wordt alles geschreven in de programmeertaal C. Je mag voor de inhoud gebruikmaken van
printf wat in vorige academiejaren niet mocht.

1. [10 pt] Het vigesimale talstelsel is een talstelsel met grondtal 20. Net zoals bij het hexadecimale talstelsel worden cijfers van 10 tot 19 voorgesteld door de letters A t.e.m. K. De letter I wordt hier niet gebruikt om geen verwarring te krijgen met het cijfer 1. De omzettingstabel van het vigesimale talstelsel is dus als volgt:
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
0 1 2 3 4 5 6 7 8 9 A B C D E F G H J K
Schrijf een C-programma dat van een aantal bestanden de vigesimale uitvoer naar het scherm schrijft. De namen van de bestanden worden via de opdrachtlijn meegegeven waarbij het minteken zoals steeds betrekking heeft op standaard invoer.
De vigesimale uitvoer bekom je door elke byte van elk invoerbestand om te zetten naar twee overeenkomstige vigesimale cijfers en die naar het scherm te schrijven. Alle vigesimale tekens worden na mekaar naar het scherm geschreven. Wanneer de gebruiker op de opdrachtlijn “./1 bestand1 bestand2” intikt, wordt in de veronderstelling dat deze bestanden bestaan, de inhoud van de bestanden bestand1 en bestand2 in vigesimale vorm naar het scherm geschreven. Wanneer een bestand niet bestaat, wordt er een foutboodschap getoond. 
**Het gebruik van streams (behalve dan printf) is NIET toegelaten!**
Schrijf de code weg in het bestand 1.c.

Wij zullen dit evengoed in bash programmeren!

In [ ]:
%%bash
#!/bin/bash

conv=( 0 1 2 3 4 5 6 7 8 9 a b c d e f )

while read -N1 kar ; do
    byte=$(printf "%d" "'$kar")
    cijfer1=$((byte/16))
    cijfer2=$((byte%16))
    echo -n ${conv[$cijfer1]}${conv[$cijfer2]}
done < "$1" # om in notebook te testen: "$1" vervangen "/etc/passwd"

**BIS**
We zullen zelf een programma schrijven in bash dat de grootste gemene deler van twee getallen berekent.

In [44]:
%%bash
#!/bin/bash

array=()
while IFS=: read getal priemfactoren ; do
    temp=()
    for getal in $priemfactoren ; do
        ((temp[$getal]++))
    done
    for el in ${!temp[@]} ; do
        [[ -z ${array[$el]} ]] && array[$el]=${temp[$el]} || {
            array[$el]="${array[$el]} ${temp[$el]}"
        }
        #echo $el ${array[$el]}
    done
done < <(factor $@)

ggd=1
for i in "${!array[@]}" ; do
    teller=0
    for j in ${array[$i]} ; do
        [[ -z $min ]] && min=$j || (($j<min)) && min=$j
        ((teller+=j))
    done
    ((teller==$#)) && ((ggd*=i**min))
done
echo $ggd

Op het examen zou iets gelijkaardigs gevraagd kunnen worden, maar met het kleinste gemene veelvoud of de grootste gemene deler van meerdere getallen.